In [1]:
import numpy as np
# import Layers as ly
# from Layers import Layer, Trainable
from tqdm import *

from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')
mnist

{'COL_NAMES': ['label', 'data'],
 'DESCR': 'mldata.org dataset: mnist-original',
 'data': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 'target': array([0., 0., 0., ..., 9., 9., 9.])}

In [2]:
import numpy as np


#----------------------------------------------------------------


class GlobalVariables(object):
    def __init__(self):
        self.trainables = []

_GLOBALS = GlobalVariables()


#----------------------------------------------------------------


class Layer(object):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def forward(self, **kwargs):
        raise Exception("This is pure virtual method")

    def backward(self, delta):
        raise Exception("This is pure virtual method")



class Trainable(object):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._variables = {}
        self._gradients = {}
        _GLOBALS.trainables.append(self)

    def train(self):
        for i in self._variables:
            # print(i, self._variables[i].shape, self._gradients[i].shape)
            self._variables[i] -= self._gradients[i]


#----------------------------------------------------------------


class FullyConnected(Layer, Trainable):
    def __init__(self, prev, size, init_fun=np.random.uniform, init_args={"low":-0.05, "high":0.05}):
        super().__init__()
        self.x = None
        self._prev = prev
        self._variables["weights"] = init_fun(**init_args, size=[prev.shape[-1], size])
        self.shape = tuple([prev.shape[0], size])
        self._gradients["weights"] = np.zeros(shape=[prev.shape[-1], size])
        
    def forward(self, **kwargs):
        self.x = self._prev.forward(**kwargs)
        self.x = self.x @ self._variables["weights"]
        return self.x
    
    def backward(self, delta):
        self_delta = delta @ self._variables["weights"].T
        self._gradients["weights"] = self._prev.x.T @ delta
        self._prev.backward(self_delta)


#----------------------------------------------------------------


class Placeholder(Layer):
    def __init__(self, sizes):
        self.x = None
        self.shape = tuple(sizes)
        
    def forward(self, **kwargs):
        return self.x
    
    def backward(self, delta):
        pass
    
    def feed(self, data):
        if data.shape[1:] != self.shape[1:]:
            raise Exception("Bad sizes")
        self.x = data



class Dropout(Layer):
    def __init__(self, prev, keep_prob=1.0):
        self.x = None
        self._prev = prev
        self._keep_prob = keep_prob
        self.shape = tuple(prev.shape)
        self._filter = None
        
    def forward(self, train=False, **kwargs):
        self.x = self._prev.forward(train=train, **kwargs)
        if train:
            self._filter = np.random.binomial(1, 1-self._keep_prob, size=self.x.shape)
            self.x[self._filter] = 0
            self.x /= self._keep_prob
        return self.x
    
    def backward(self, delta):
        self_delta = delta * self._keep_prob
        self_delta[self._filter] = 0
        self._prev.backward(self_delta)



class ReLU(Layer):
    def __init__(self, prev):
        self.x = None
        self._prev = prev
        self.shape = tuple(prev.shape)
        
    def forward(self, **kwargs):
        self.x = self._prev.forward(**kwargs)
        self.x *= (self.x > 0).astype(np.int8)
        return self.x
    
    def backward(self, delta):
        self_delta = delta*(self._prev.x > 0).astype(np.int8)
        self._prev.backward(self_delta)



class Sigmoid(Layer):
    def __init__(self, prev):
        self.x = None
        self._prev = prev
        self.shape = tuple(prev.shape)
    
    def forward(self, **kwargs):
        self.x = self._prev.forward(**kwargs)
        self.x = 1/(1 + np.exp(-self.x))
        return self.x
    
    def backward(self, delta):
        self_delta = self.x*(1-self.x)*delta
        self._prev.backward(self_delta)



class Softmax(Layer):
    def __init__(self, prev):
        self.x = None
        self._prev = prev
        self.shape = tuple(prev.shape)
        
    def forward(self, **kwargs):
        self.x = self._prev.forward(**kwargs)
        self.x = np.exp(self.x)
        self.x /= np.sum(self.x, -1)[:,None]
        return self.x
    
    def backward(self, delta):
        self_delta = np.zeros(delta.shape)
        for i in range(self_delta.shape[1]):
            for k in range(self_delta.shape[1]):
                self_delta[:, i] += delta[:,k]*(self.x[:,i]*(1-self.x[:,i]) if i==k else -self.x[:,i]*self.x[:,k])
        self._prev.backward(self_delta)

#----------------------------------------------------------------

class SoftmaxCrossEntropy(object):
    def __init__(self, predicts, labels, learn_rate=0.001):
        if predicts.shape[1:] != labels.shape[1:]:
            raise Exception("Bad sizes")
        self._predicts = predicts
        self._labels = labels
        self._learn_rate = learn_rate
        
    @staticmethod
    def _softmax(x):
        s = np.exp(x)
        s /= np.sum(s, -1)[:,None]
        return s*(1-1e-8) # to not obtain 0 or 1 for numerical stable
    
    @staticmethod
    def _cross_entropy(s, y):
        loss = -np.sum(y*np.log(s) + (1-y)*np.log(1-s))/y.shape[0]
        return np.nan_to_num(loss)
    
    def run(self):
        y = self._labels.forward()
        x = self._predicts.forward(train=True)
        s = SoftmaxCrossEntropy._softmax(x)
        delta = -y + (1-y)*s/(1-s)
        for k in range(delta.shape[1]):
            if s[:,k].max() == 1:
                print(s[:,k])
            delta[:,:] += (y-s)[:,k][:,None] * (s/(1-s[:,k][:,None]))
        self._predicts.backward((1/y.shape[0])*np.nan_to_num(delta)*self._learn_rate)

        for trainable in _GLOBALS.trainables:
            trainable.train()

        return SoftmaxCrossEntropy._cross_entropy(s, y)



class SigmoidCrossEntropy(object):
    def __init__(self, predicts, labels, learn_rate=0.001):
        if predicts.shape[1:] != labels.shape[1:]:
            raise Exception("Bad sizes")
        self._predicts = predicts
        self._labels = labels
        self._learn_rate = learn_rate
        
    @staticmethod
    def _sigmoid(x):
        sigma = 1/(1 + np.exp(-x))
        return sigma*(1-1e-8) # to not obtain 0 or 1 for numerical stable
    
    @staticmethod
    def _cross_entropy(sigma, y):
        loss = -np.sum(y*np.log(sigma) + (1-y)*np.log(1-sigma))/y.shape[0]
        return np.nan_to_num(loss)
    
    def run(self):
        y = self._labels.forward()
        x = self._predicts.forward(train=True)
        sigma = SigmoidCrossEntropy._softmax(x)
        delta = sigma - y
        self._predicts.backward((1/y.shape[0])*np.nan_to_num(delta)*self._learn_rate)

        for trainable in _GLOBALS.trainables:
            trainable.train()

        return SigmoidCrossEntropy._cross_entropy(sigma, y)


In [3]:
class BatchNormalization(Layer, Trainable):
    def _checkParams(self, dims, prevshape):
        if type(dims) is int:
            dims = tuple([dims])
        else:
            dims = tuple(np.sort(dims))
        if len(prevshape) < dims[-1]+1:
            raise Exception("x has to few dimensions")
        self._dims = dims
        prevshape = list(prevshape[:])
        for d in dims:
            prevshape[d] = 1
        self._statshape = tuple(prevshape[:])
        prevshape[0] = 1
        return prevshape
    
    
    def __init__(self, prev, dims, epsi=1e-8, ema=0.99, init_fun=np.random.uniform, init_args={"low":-0.05, "high":0.05}):
        super().__init__()
        varshape = self._checkParams(dims, prev.shape)
        self.x = None
        self._prev = prev
        self.shape = prev.shape[:]
        self._variables["gamma"] = init_fun(**init_args, size=varshape) + 1
        self._variables["beta"] = init_fun(**init_args, size=varshape)
        self._gradients["gamma"] = np.zeros(shape=varshape) + 1
        self._gradients["beta"] = np.zeros(shape=varshape)
        self._validvars = {}
        self._validvars["mu"] = np.zeros(shape=varshape) + 1
        self._validvars["sigma"] = np.zeros(shape=varshape)
        self._epsi = epsi
        self._ema = ema
        self._d_dx = None
        self._d_dg = None
        # print(self._dims, self._variables["gamma"].shape, self._gradients["gamma"].shape)
        
    def forward(self, train=False, **kwargs):
        self.x = self._prev.forward(train=train, **kwargs)
        # print("train", train)
        if train:
            mu = self.x.mean(self._dims).reshape(self._statshape)
            sigma = self.x.var(self._dims).reshape(self._statshape)
            # print("mu, simg", mu.shape, sigma.shape)
            cntr = (self.x - mu)
            denom = np.sqrt(sigma + self._epsi)
            # print("cntr, denom", cntr.shape, denom.shape)
            self._d_dx = self._variables["gamma"]/denom
            self._d_dg = np.expand_dims(cntr.mean(0), 0)/denom
            # print("self._d_dx, self._d_dg", self._d_dx.shape, self._d_dg.shape)
            self.x = self._d_dx*cntr + self._variables["beta"]
            if 0 not in self._dims:
                mu = np.expand_dims(mu.mean(0), 0)
                sigma = np.expand_dims(sigma.mean(0), 0)
                self._d_dx = np.expand_dims(self._d_dx.mean(0), 0)
                self._d_dg = np.expand_dims(self._d_dg.mean(0), 0)
            self._validvars["mu"] = self._ema*self._validvars["mu"] + (1-self._ema)*mu
            self._validvars["sigma"] = self._ema*self._validvars["sigma"] + (1-self._ema)*sigma
        else:
            # print("test")
            a = self._variables["gamma"]/np.sqrt(self._validvars["sigma"] + self._epsi)
            self.x = a*(self.x - self._validvars["mu"]) + self._variables["beta"]
        return self.x
    
    def backward(self, delta):
        self_delta = delta * self._d_dx
        delta = np.mean(delta, self._dims).reshape(self._gradients["beta"].shape)
        self._gradients["beta"] = delta
        self._gradients["gamma"] = delta * self._d_dg
        # print(self._dims, self._variables["gamma"].shape, self._gradients["gamma"].shape, self._d_dg.shape)
        # print(self._dims, self._variables["beta"].shape, self._gradients["beta"].shape, self._d_dg.shape)
        self._prev.backward(self_delta)

In [4]:
X, y = mnist["data"], mnist["target"]
y_ = y.astype(int)
y = np.zeros([y_.shape[0], 10])
y[np.arange(y_.shape[0]), y_] = 1
X = X.astype(np.float32)/255
shuffle_index = np.random.permutation(X.shape[0])
X, y = X[shuffle_index], y[shuffle_index]

In [5]:
def test(imgs, lbls, last, X, y, bs=-1):
    if bs == -1:
        bs = y.shape[0]
    score = 0
    for i in range(int(X.shape[0]/bs)):
        imgs.feed(X[i*bs:(i+1)*bs])
        preds = last.forward();
        res = np.argmax(preds, -1)
        lbls = np.argmax(y[i*bs:(i+1)*bs], -1)
        score += np.sum(res == lbls)
    return score/y.shape[0]
        
#     imgs.feed(X)
#     preds = last.forward();
#     res = np.argmax(preds, -1)
#     lbls = np.argmax(y, -1)
#     return np.sum(res == lbls)/lbls.shape[0]
    
def train(imgs, lbls, cost, n_ep, X, y, bs):
    for e in range(n_ep):
        shuffle_index = np.random.permutation(X.shape[0])
        X, y = X[shuffle_index], y[shuffle_index]
        loss = []
        for i in range(int(X.shape[0]/bs)):
        #for i in range(int(X.shape[0]/bs)):
            imgs.feed(X[i*bs:(i+1)*bs])
            lbls.feed(y[i*bs:(i+1)*bs])
            loss.append(cost.run())
        print("\repoch {0} done; mean loss = {1:.5f}".format(e+1, np.array(loss).mean()))

In [6]:
SHAPES = [800, 200]

# BatchNormalization before 2nd

In [7]:
imgs = Placeholder([-1, 784])
lbls = Placeholder([-1, 10])

x = FullyConnected(imgs, SHAPES[0])
x = Dropout(x, 0.9)
x = ReLU(x)
x = BatchNormalization(x, [0])
x = FullyConnected(x, SHAPES[1])
x = Dropout(x, 0.9)
x = ReLU(x)
# x = BatchNormalization(x, [0])
x = FullyConnected(x, 10)

s = Softmax(x)
crsEnt = SoftmaxCrossEntropy(x, lbls, 0.05)

In [8]:
train(imgs, lbls, crsEnt, 10, X[:55000], y[:55000], 128)

epoch 1 done; mean loss = 0.55612
epoch 2 done; mean loss = 0.25858
epoch 3 done; mean loss = 0.19741
epoch 4 done; mean loss = 0.16194
epoch 5 done; mean loss = 0.13894
epoch 6 done; mean loss = 0.11881
epoch 7 done; mean loss = 0.10367
epoch 8 done; mean loss = 0.09440
epoch 9 done; mean loss = 0.08591
epoch 10 done; mean loss = 0.07962


In [9]:
batch_sizes = [1,4,128,-1]
for bs in batch_sizes:
    print("batch size:", bs if bs > 0 else "inf")
    print("\ttrain score:", test(imgs, lbls, s, X[:55000], y[:55000], bs))
    print("\tvalid score:", test(imgs, lbls, s, X[55000:65000], y[55000:65000], bs))
    print("\ttest score: ", test(imgs, lbls, s, X[65000:], y[65000:], bs))

batch size: 1
	train score: 0.9966
	valid score: 0.9777
	test score:  0.9794
batch size: 4
	train score: 0.9966
	valid score: 0.9777
	test score:  0.9794
batch size: 128
	train score: 0.9950181818181818
	valid score: 0.9761
	test score:  0.9778
batch size: inf
	train score: 0.9966
	valid score: 0.9777
	test score:  0.9794


# BatchNormalization before 3rd

In [10]:
imgs = Placeholder([-1, 784])
lbls = Placeholder([-1, 10])

x = FullyConnected(imgs, SHAPES[0])
x = Dropout(x, 0.9)
x = ReLU(x)
# x = BatchNormalization(x, [0])
x = FullyConnected(x, SHAPES[1])
x = Dropout(x, 0.9)
x = ReLU(x)
x = BatchNormalization(x, [0])
x = FullyConnected(x, 10)

s = Softmax(x)
crsEnt = SoftmaxCrossEntropy(x, lbls, 0.05)

In [11]:
train(imgs, lbls, crsEnt, 10, X[:55000], y[:55000], 128)

epoch 1 done; mean loss = 0.59711
epoch 2 done; mean loss = 0.33537
epoch 3 done; mean loss = 0.26523
epoch 4 done; mean loss = 0.22394
epoch 5 done; mean loss = 0.19512
epoch 6 done; mean loss = 0.17535
epoch 7 done; mean loss = 0.15707
epoch 8 done; mean loss = 0.14490
epoch 9 done; mean loss = 0.13193
epoch 10 done; mean loss = 0.12268


In [12]:
batch_sizes = [1,4,128,-1]
for bs in batch_sizes:
    print("batch size:", bs if bs > 0 else "inf")
    print("\ttrain score:", test(imgs, lbls, s, X[:55000], y[:55000], bs))
    print("\tvalid score:", test(imgs, lbls, s, X[55000:65000], y[55000:65000], bs))
    print("\ttest score: ", test(imgs, lbls, s, X[65000:], y[65000:], bs))

batch size: 1
	train score: 0.9917636363636364
	valid score: 0.9765
	test score:  0.98
batch size: 4
	train score: 0.9917636363636364
	valid score: 0.9765
	test score:  0.98
batch size: 128
	train score: 0.9901818181818182
	valid score: 0.975
	test score:  0.9786
batch size: inf
	train score: 0.9917636363636364
	valid score: 0.9765
	test score:  0.98


# BatchNormalization before all hiden

In [13]:
imgs = Placeholder([-1, 784])
lbls = Placeholder([-1, 10])

x = FullyConnected(imgs, SHAPES[0])
x = Dropout(x, 0.9)
x = ReLU(x)
x = BatchNormalization(x, [0])
x = FullyConnected(x, SHAPES[1])
x = Dropout(x, 0.9)
x = ReLU(x)
x = BatchNormalization(x, [0])
x = FullyConnected(x, 10)

s = Softmax(x)
crsEnt = SoftmaxCrossEntropy(x, lbls, 0.05)

In [14]:
train(imgs, lbls, crsEnt, 10, X[:55000], y[:55000], 128)

epoch 1 done; mean loss = 0.49949
epoch 2 done; mean loss = 0.25034
epoch 3 done; mean loss = 0.19095
epoch 4 done; mean loss = 0.15934
epoch 5 done; mean loss = 0.13713
epoch 6 done; mean loss = 0.12167
epoch 7 done; mean loss = 0.10949
epoch 8 done; mean loss = 0.10010
epoch 9 done; mean loss = 0.09117
epoch 10 done; mean loss = 0.08676


In [15]:
batch_sizes = [1,4,128,-1]
for bs in batch_sizes:
    print("batch size:", bs if bs > 0 else "inf")
    print("\ttrain score:", test(imgs, lbls, s, X[:55000], y[:55000], bs))
    print("\tvalid score:", test(imgs, lbls, s, X[55000:65000], y[55000:65000], bs))
    print("\ttest score: ", test(imgs, lbls, s, X[65000:], y[65000:], bs))

batch size: 1
	train score: 0.9962181818181818
	valid score: 0.9791
	test score:  0.981
batch size: 4
	train score: 0.9962181818181818
	valid score: 0.9791
	test score:  0.981
batch size: 128
	train score: 0.9946181818181818
	valid score: 0.9775
	test score:  0.9798
batch size: inf
	train score: 0.9962181818181818
	valid score: 0.9791
	test score:  0.981


# No BatchNormalization

In [16]:
imgs = Placeholder([-1, 784])
lbls = Placeholder([-1, 10])

x = FullyConnected(imgs, SHAPES[0])
x = Dropout(x, 0.9)
x = ReLU(x)
# x = BatchNormalization(x, [0])
x = FullyConnected(x, SHAPES[1])
x = Dropout(x, 0.9)
x = ReLU(x)
# x = BatchNormalization(x, [0])
x = FullyConnected(x, 10)

s = Softmax(x)
crsEnt = SoftmaxCrossEntropy(x, lbls, 0.05)

In [17]:
train(imgs, lbls, crsEnt, 10, X[:55000], y[:55000], 128)

epoch 1 done; mean loss = 1.29547
epoch 2 done; mean loss = 0.52528
epoch 3 done; mean loss = 0.41337
epoch 4 done; mean loss = 0.34510
epoch 5 done; mean loss = 0.29641
epoch 6 done; mean loss = 0.25935
epoch 7 done; mean loss = 0.23135
epoch 8 done; mean loss = 0.20839
epoch 9 done; mean loss = 0.18902
epoch 10 done; mean loss = 0.17362


In [18]:
batch_sizes = [1,4,128,-1]
for bs in batch_sizes:
    print("batch size:", bs if bs > 0 else "inf")
    print("\ttrain score:", test(imgs, lbls, s, X[:55000], y[:55000], bs))
    print("\tvalid score:", test(imgs, lbls, s, X[55000:65000], y[55000:65000], bs))
    print("\ttest score: ", test(imgs, lbls, s, X[65000:], y[65000:], bs))

batch size: 1
	train score: 0.9835454545454545
	valid score: 0.972
	test score:  0.973
batch size: 4
	train score: 0.9835454545454545
	valid score: 0.972
	test score:  0.973
batch size: 128
	train score: 0.9819636363636364
	valid score: 0.9705
	test score:  0.9716
batch size: inf
	train score: 0.9835454545454545
	valid score: 0.972
	test score:  0.973


# ------------------------------------------------

In [19]:
def normalize(x, dims, epsi=1e-8):
    if type(d) is int:
        d = tuple([d])
    else:
        dims = tuple(np.sort(dims))
    if x.ndim < np.max(dims)+1:
        raise Exception("x has to few dimensions")
    mu = x.mean(dims)
    sigm = x.var(dims)
    for d in dims:
        mu = np.expand_dims(mu, d)
        sigm = np.expand_dims(sigm, d)
    return (x - mu)/np.sqrt(sigm + epsi)